In [ ]:
!pip install boto3 langchain langchain_community -U

In [ ]:
import json
import boto3
import random

In [ ]:
bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
filename = 'data_result.csv'
# df = pd.read_csv('data_result.csv',nrows=1000)
with open(filename) as f:
    lines = f.readlines()

In [ ]:
# 设置要选取的连续元素的长度
n = 10

# 随机选取一个起始索引
start_idx = random.randint(0, len(lines) - n)
lines[start_idx:start_idx+n]

In [ ]:
import re
extracted_data = []

# 提取聊天的正则表达式模式
pattern = r'"m":"(.*?)"'

# 过滤出link

# 遍历每一行
for line in lines:
    match = re.search(pattern, line)
    if match:
        text = match.group(1)
        if not text.startswith('link'):
            extracted_data.append(text)

In [ ]:
len(extracted_data)

In [ ]:

# 随机选取一个起始索引
def random_cut_arr(arr,n,seed=1):
    random.seed(seed)
    start_idx = random.randint(0, len(arr) - n)
    return arr[start_idx:start_idx+n]

In [ ]:
random_cut_arr(extracted_data,10)

In [ ]:
cut_extracted_data = random_cut_arr(extracted_data,500)
text = "\n".join(cut_extracted_data)

In [ ]:
len(text)

In [ ]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate

In [ ]:
llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [ ]:
template_extract = \
"""You are an expert research assistant, tasked with identifying player sentiments regarding certain in-game items, neutral NPCs, and game market activities. 

Here is a document you will analyze
<doc>
{context}
</doc>

Here is a task:
First, find the quotes from the document that are most relevant to {topic}, and then print them in numbered order. Quotes should be relatively short.
If there are no relevant quotes, write "No relevant quotes" instead.
please enclose your analysis results in xml tag <response>.

for example:
<response>
1. "拍卖行多香" 
2. "我拍到好东西了" 
3. "拍卖行太差劲了" 
4. "auction sucks" 
5. "拍卖行有人发包" 
</response>

Skip the preamble, go straight into the answer.
"""

In [ ]:
template_sentiment = \
"""You are a chat message sentiment classifer

Here is a document you will classify the senetiment
<doc>
{relevant_info}
</doc>


please list all the content if it is relevant to {topic} and classify the sentiment of each content into [positive,neutral,negative]'

Please follow below requirements:
1. You will strictly be based on the document in <doc>.
2. please enclose your analysis results in xml tag <sentiment>.

for example:
<sentiment>
1. "拍卖行多香" [positive]
2. "我拍到好东西了" [positive]
3. "拍卖行太差劲了" [negative]
4. "auction sucks" [negative]
5. "拍卖行有人发包" [neutral]
</sentiment>

Skip the preamble, go straight into the answer. 

"""

In [ ]:
from langchain_core.output_parsers.base import BaseOutputParser
import re
class CustOuputParser(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<response>(.*?)</response>"
        match = re.search(pattern, content, re.DOTALL)
        if match:
            text = match.group(1)
            text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
            return text
        else:
            return 'No relevant quotes'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"


class CustOuputParser2(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<sentiment>(.*?)</sentiment>"
        match = re.search(pattern, content, re.DOTALL)
        return match.group(1) if match else 'No sentiment'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"
    

In [ ]:
def split_into_chunks(arr, n):
    chunks = []
    for i in range(0, len(arr), n):
        chunks.append(arr[i:i+n])
    return chunks

In [ ]:
chunks = split_into_chunks(extracted_data,2000)

In [ ]:
print(f"total chunks {len(chunks)}")

In [ ]:
text =  "\n".join(chunks[45])
len(text)

In [ ]:
# from langchain_core.output_parsers import StrOutputParser
# prompt = ChatPromptTemplate.from_template(template)
prompt_extract = ChatPromptTemplate.from_template(template_extract)
prompt_sentiment = ChatPromptTemplate.from_template(template_sentiment)

In [ ]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
## 如果没有相关内容则无须invoke chain2
def route(info):
    if not 'no relevant quotes' in info['relevant_info'].lower():
        return chain_2

In [ ]:
output_parser = CustOuputParser()
output2_parser  = CustOuputParser2()

chain_1 = prompt_extract | llm_sonnet |output_parser
chain_2 = prompt_sentiment | llm_sonnet|output2_parser
# chain_3 = ({'relevant_info':chain_1,'topic':itemgetter('topic')})|prompt_sentiment | llm_haiku|output2_parser


In [ ]:
##测试chain3
# chain_3.invoke({'topic':"auction house",
#                        'context':text})

In [ ]:
##测试加入了路由选择的chainfull
full_chain = ({'relevant_info':chain_1,'topic':itemgetter('topic')})| RunnableLambda(route)
answer = full_chain.invoke({'topic':"auction house",'context':text})

In [ ]:
answer

In [ ]:
# question = "is there any content relevant to auction house?"

#'please list all the content if it is relevant and classify the sentiment of each content into [positive,neutral,negative]'

# answer = chain_2.invoke({'topic':"auction house",
#                        'context':text})

In [ ]:
# print(answer)

In [ ]:
def extract_text_and_sentiment(text) -> list:
    pattern = r'^(.*?)\s*\[(.*?)\]'
    # pattern = r'"(.*?)"\s*\[(.*?)\]$'
    if not text:
        return []
    result = []
    for line in text.split('\n'):
        match = re.match(pattern, line)
        if match:
            # print(match)
            text_part = match.group(1)
            quoted_texts = re.findall(r'"(.*?)"', text_part)
            # print(quoted_texts)
            sentiment_part = match.group(2)
            result.append((quoted_texts[0], sentiment_part))
    return result

In [ ]:
extract_text_and_sentiment(answer)

In [ ]:
#save a list to a local file, if the file is exist then append the list to the file
def save_list(list,file_name):
    with open(file_name,'a') as f:
        for item in list:
            f.write("%s\n" % item)
    f.close()
    return True

In [ ]:
#save a list to a csv file using pandas, if the file is exist then append the list to the file
import pandas as pd
import os
def save_to_csv(data, filename):
    new_data = pd.DataFrame(data, columns=['text','sentiment'])
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        #append data to df
        df = pd.concat([df, new_data], ignore_index=True)
    else:
        df = new_data
    df.to_csv(filename, index=False)
    return df

循环每个chunk进行输出,每次追加到csv文件保存，

In [ ]:
import time
all_result = []
filename = 'parsed_result'

##断点跳过
skip_num =238
for i,chunk in enumerate(chunks[:1000]):
    if i < skip_num:
        continue
    t1 = time.time()
    print(f'--------chunk idx:{i}-------')
    text =  "\n".join(chunk)
    answer = full_chain.invoke({'topic':"auction house",
                       'context':text})
    extract_ret = extract_text_and_sentiment(answer)
    print(f'\nextract_ret:\n{extract_ret}')
    time.sleep(2)
    print(f'---time cost {time.time()-t1} s -----\nsleep 10s for token tpm\n\n')
    if extract_ret:
        all_result+=extract_ret
        save_to_csv(extract_ret,filename+f"chunk_{i}.csv")
        save_to_csv(extract_ret,filename+f"_all.csv")

In [ ]:
len(all_result)